In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#Opção 
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#Calculo do KS 
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [2]:
##  create a dataframe
df = pd.read_csv("Base_modelagem_3.csv", sep=";")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511721 entries, 0 to 511720
Data columns (total 10 columns):
PAGTO                           511721 non-null int64
GRP_PRAZO_ULTIMACOMPRA_03M_2    511721 non-null object
GRP_QTDE_COMPRAS_ULT03M_2       511721 non-null object
GRP_QTDE_COMPULT03M_OFFUS_2     511721 non-null object
GRP_TMP_FAT_ANTERIOR_M1_2       511721 non-null object
GRP_QTD_SLDOCRESC_06M_2         511721 non-null object
GRP_FATURAATUALSLDTOTAL_M0_2    511721 non-null object
GRP_FATURAATUALFATAVG_3M_2      511721 non-null object
GRP_QTD_ROTATIVOU_12M_2         511721 non-null object
GRP_QTD_FATURAS_06M_2           511721 non-null object
dtypes: int64(1), object(9)
memory usage: 39.0+ MB


In [3]:
df2=pd.get_dummies(df, prefix=['col1', 'col2', 'col3','col4','col5','col6', 'col7','col8','col9'])

In [4]:
df2.head(5)

,PAGTO,"col1_(-Inf,3]","col1_(3, Inf]",col1_Missing,"col2_(-Inf,Inf]",col2_Missing,"col3_(-Inf,9]","col3_(9, Inf]",col3_Missing,"col4_(-Inf,2]","col4_(2, Inf]",col4_Missing,col5_0,col5_1,col5_2,col5_3,col5_4,col5_missing,col6_1,col6_2,col6_3,col6_4,col6_missing,col7_1,col7_2,col7_3,col7_4,col7_5,col7_6,col7_missing,"col8_(-Inf,0]","col8_(0,1]","col8_(1, Inf]]",col8_Missing,"col9_(-Inf,3]","col9_(3, Inf]]",col9_Missing
0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0
1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
2,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0
3,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0
4,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0


In [6]:
df2=df2.rename({'col1_(-Inf,3]':'col1_a', 	'col1_(3, Inf]':'col1_b', 	'col1_Missing':'col1_c', 	'col2_(-Inf,Inf]':'col2_a', 	'col2_Missing':'col2_b', 	'col3_(-Inf,9]':'col3_a', 	'col3_(9, Inf]':'col3_b', 	'col3_Missing':'col3_c', 	'col4_(-Inf,2]':'col4_a', 	'col4_(2, Inf]':'col4_b', 	'col4_Missing':'col4_c', 	'col5_0':'col5_a', 	'col5_1':'col5_b', 	'col5_2':'col5_c', 	'col5_3':'col5_d', 	'col5_4':'col5_e', 	'col5_missing':'col5_f', 	'col6_1':'col6_a', 	'col6_2':'col6_b', 	'col6_3':'col6_c', 	'col6_4':'col6_d', 	'col6_missing':'col6_e', 	'col7_1':'col7_a', 	'col7_2':'col7_b', 	'col7_3':'col7_c', 	'col7_4':'col7_d', 	'col7_5':'col7_e', 	'col7_6':'col7_f', 	'col7_missing':'col7_g', 	'col8_(-Inf,0]':'col8_a', 	'col8_(0,1]':'col8_b', 	'col8_(1, Inf]]':'col8_c', 	'col8_Missing':'col8_d', 	'col9_(-Inf,3]':'col9_a', 	'col9_(3, Inf]]':'col9_b', 	'col9_Missing':'col9_c', }, axis='columns')


In [7]:
df2.isnull().sum()

PAGTO     0
col1_a    0
col1_b    0
col1_c    0
col2_a    0
col2_b    0
col3_a    0
col3_b    0
col3_c    0
col4_a    0
col4_b    0
col4_c    0
col5_a    0
col5_b    0
col5_c    0
col5_d    0
col5_e    0
col5_f    0
col6_a    0
col6_b    0
col6_c    0
col6_d    0
col6_e    0
col7_a    0
col7_b    0
col7_c    0
col7_d    0
col7_e    0
col7_f    0
col7_g    0
col8_a    0
col8_b    0
col8_c    0
col8_d    0
col9_a    0
col9_b    0
col9_c    0
dtype: int64

In [8]:
corr = df2.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# teste

In [9]:
for i in range(0,len(df2.columns)):
    var=df2[df2.columns[i]].value_counts() 
    print(df2.columns[i])
    print(var)

PAGTO
0    432707
1     79014
Name: PAGTO, dtype: int64
col1_a
0    445429
1     66292
Name: col1_a, dtype: int64
col1_b
0    451893
1     59828
Name: col1_b, dtype: int64
col1_c
1    385601
0    126120
Name: col1_c, dtype: int64
col2_a
0    385601
1    126120
Name: col2_a, dtype: int64
col2_b
1    385601
0    126120
Name: col2_b, dtype: int64
col3_a
0    445080
1     66641
Name: col3_a, dtype: int64
col3_b
0    443847
1     67874
Name: col3_b, dtype: int64
col3_c
1    377206
0    134515
Name: col3_c, dtype: int64
col4_a
1    431268
0     80453
Name: col4_a, dtype: int64
col4_b
0    474414
1     37307
Name: col4_b, dtype: int64
col4_c
0    468575
1     43146
Name: col4_c, dtype: int64
col5_a
0    467029
1     44692
Name: col5_a, dtype: int64
col5_b
0    451792
1     59929
Name: col5_b, dtype: int64
col5_c
0    380578
1    131143
Name: col5_c, dtype: int64
col5_d
0    349779
1    161942
Name: col5_d, dtype: int64
col5_e
0    398535
1    113186
Name: col5_e, dtype: int64
col5_f
0    5108

In [121]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
import Calc_KS as ks

In [165]:
df_z= df2.loc[:,['PAGTO','col1_a', 	'col1_b']]

In [175]:
df_z= df2.loc[:,['PAGTO','col1_a', 	'col1_b', 	'col1_c', 	'col2_a', 	'col2_b', 	'col3_a']]

In [171]:
df_z= df2.loc[:,['PAGTO','col1_a', 	'col1_b', 	'col1_c', 	'col2_a', 	'col2_b', 	'col3_a', 	'col3_b', 	'col3_c', 	'col4_a', 	'col4_b', 	'col4_c', 	'col5_a', 	'col5_b', 	'col5_c', 	'col5_d', 	'col5_e', 	'col5_f', 	'col6_a', 	'col6_b', 	'col6_c', 	'col6_d', 	'col6_e', 	'col7_a', 	'col7_b', 	'col7_c', 	'col7_d', 	'col7_e', 	'col7_f', 	'col7_g', 	'col8_a', 	'col8_b', 	'col8_c', 	'col8_d', 	'col9_a', 	'col9_b', 	'col9_c']]

In [172]:
df_z= df_z.sample(100000) 

In [173]:
# Tirar a primeira columa (resposta)
colunas=df_z.columns[1:]

In [174]:
vare = "PAGTO ~ col1_a"

for number in colunas[1:]:
        #gera a formula 
        vare = vare + '+' + number
        formula=vare
        print('----------------------')
        print(number)
        print(vare)
        
        #gera o modelo  
        model = smf.glm(formula =formula , data=df_z, family=sm.families.Binomial())
        result = model.fit()
        print(result.summary())
        print('----------------------')
        print('----------------------')
        
        #scorar a base 
        #Tirar o intercepto
        betas=result.params[1:]
        len(betas)
        
        #score
        df_z['score']=df_z[colunas[0]]*0+result.params[0]
        for i in range(0,len(betas)):
            df_z['score']= df_z['score']+betas[i]*df_z[colunas[i]]
    
        df_z['score_1']=round((np.exp(df_z['score']) /(1+np.exp(df_z['score'])))*1000,0)
        
        ## Ks
        try:
            mydf=ks(data=df_z,target="PAGTO", prob="score_1")
            print(mydf)
        except ValueError:
            print("não foi possivel calcular o KS")
            
        print('----------------------')
        test=stats.ks_2samp(df_z['score_1'], df_z['PAGTO'])
        print('KS de:')
        print(test) 
        df_z=df_z.drop(['target0', 'score_1', 'score' ], axis=1)

----------------------
col1_b
PAGTO ~ col1_a+col1_b
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -42600.
Date:                Wed, 05 Feb 2020   Deviance:                       85200.
Time:                        23:09:03   Pearson chi2:                 1.00e+05
No. Iterations:                     5   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8160      0.011   -172.916      0.000      -1.837      -1.795


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99995
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -42056.
Date:                Wed, 05 Feb 2020   Deviance:                       84112.
Time:                        23:09:10   Pearson chi2:                 9.69e+04
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.275e+11   7.71e+11     -0.165      0.869   -1.64e+12    1.38e+12
col1_a     -2.765e+11   5.84e+11     -0.474      0.6

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99993
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -40498.
Date:                Wed, 05 Feb 2020   Deviance:                       80995.
Time:                        23:09:20   Pearson chi2:                 1.03e+05
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.129e+12   1.29e+12     -2.428      0.015   -5.66e+12   -6.03e+11
col1_a     -1.437e+12   1.77e+12     -0.814      0.4

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99989
Model Family:                Binomial   Df Model:                           10
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -40175.
Date:                Wed, 05 Feb 2020   Deviance:                       80351.
Time:                        23:09:33   Pearson chi2:                 9.77e+04
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.742e+11   1.25e+12     -0.300      0.764   -2.82e+12    2.07e+12
col1_a      1.778e+10   5.92e+10      0.300      0.7

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99987
Model Family:                Binomial   Df Model:                           12
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -34067.
Date:                Wed, 05 Feb 2020   Deviance:                       68135.
Time:                        23:09:47   Pearson chi2:                 1.02e+05
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -1.29e+11    4.2e+11     -0.307      0.759   -9.53e+11    6.95e+11
col1_a      1.019e+11   3.32e+11      0.307      0.7

...

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99975
Model Family:                Binomial   Df Model:                           24
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 05 Feb 2020   Deviance:                          nan
Time:                        23:11:23   Pearson chi2:                 6.82e+19
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.766e+24   1.03e+19  -4.64e+05      0.000   -4.77e+24   -4.77e+24
col1_a      6.175e+25   3.17e+19   1.95e+06      0.0

10           0.0      11.0     263       9951      1.71%        11.76%       100.00%          100.00%   0.0
KS is 62.4% at decile 3
        min_prob  max_prob  events  nonevents event_rate nonevent_rate cum_eventrate cum_noneventrate    KS
Decile                                                                                                     
1          499.0     961.0    7515       2433     48.88%         2.88%        48.88%            2.88%  46.0
2          266.0     498.0    3047       5127     19.82%         6.06%        68.70%            8.93%  59.8
3          133.0     265.0    2161       9677     14.06%        11.44%        82.76%           20.37%  62.4
4           81.0     132.0     943       9078      6.13%        10.73%        88.89%           31.10%  57.8
5           56.0      80.0     450       9397      2.93%        11.10%        91.82%           42.20%  49.6
6           39.0      55.0     274       9263      1.78%        10.95%        93.60%           53.15%  40.5
7   

C:\Users\Jonas\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:880: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\Jonas\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:879: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\Jonas\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:880: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\Jonas\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:879: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\Jonas\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:932: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
C:\Users\Jonas\Anaconda3\lib\site-packages\

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99974
Model Family:                Binomial   Df Model:                           25
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 05 Feb 2020   Deviance:                          nan
Time:                        23:11:43   Pearson chi2:                 5.70e+19
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    4.34e+25   1.02e+19   4.24e+06      0.000    4.34e+25    4.34e+25
col1_a     -2.082e+26   3.12e+19  -6.67e+06      0.0

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99974
Model Family:                Binomial   Df Model:                           25
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 05 Feb 2020   Deviance:                          nan
Time:                        23:12:03   Pearson chi2:                 6.60e+19
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -7.893e+26   5.56e+19  -1.42e+07      0.000   -7.89e+26   -7.89e+26
col1_a     -3.747e+25   5.22e+19  -7.17e+05      0.0

In [ ]:
# df2=df2.drop(['col2_c', 'col5_e', 'col6_d','col8_c','col9_e', 'col10_d' ], axis=1)

In [27]:
len(colunas)

2

In [47]:
df3[colunas[1]]

197304    0
172607    1
419992    0
286973    0
197850    0
387237    0
346578    0
32940     0
468339    1
355714    0
348856    0
103261    0
232020    0
39834     0
181400    0
473170    0
49202     0
167731    0
126783    0
331949    0
382285    0
432770    0
240639    1
276502    0
303295    0
347714    0
346042    0
254223    0
227687    0
328102    1
322667    0
129657    0
174623    0
388786    0
341665    0
21607     0
471623    0
197404    1
436277    0
352172    1
436064    0
277588    0
124693    0
353523    0
30281     0
110491    0
166531    0
468234    0
378194    0
487659    0
469321    0
271009    0
79813     0
92925     0
474027    0
374904    0
138271    0
463691    0
383784    0
319831    0
185838    0
114207    0
331447    0
212116    0
102806    0
296421    0
262060    0
309931    0
30703     0
462387    0
366484    0
13336     0
42113     0
137760    0
388799    0
214977    0
59829     0
501398    0
115867    0
491458    0
320582    0
32960     0
33576     0
4631

In [126]:
df_z['score']=df_z[colunas[0]]*0+result.params[0]

In [139]:
df_z['score_1']=round((np.exp(df_z['score']) /(1+np.exp(df_z['score'])))*1000,0)

In [140]:
df_z.head(5)

,PAGTO,col1_a,col1_b,score,score_1,target0
0,0,0,0,-1.812453,140.0,1
1,0,1,0,-1.219088,228.0,1
2,0,0,0,-1.812453,140.0,1
3,0,0,0,-1.812453,140.0,1
4,0,0,0,-1.812453,140.0,1


In [134]:

def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [143]:
try:
    ks(data=df_z,target="PAGTO", prob="score_1")
except ValueError:
    print("não foi possivel calcular o KS")

não foi possivel calcular o KS


In [144]:
df_z

,PAGTO,col1_a,col1_b,score,score_1,target0
0,0,0,0,-1.812453,140.0,1
1,0,1,0,-1.219088,228.0,1
2,0,0,0,-1.812453,140.0,1
3,0,0,0,-1.812453,140.0,1
4,0,0,0,-1.812453,140.0,1
5,0,1,0,-1.219088,228.0,1
6,0,0,0,-1.812453,140.0,1
7,0,0,0,-1.812453,140.0,1
8,0,1,0,-1.219088,228.0,1
9,0,1,0,-1.219088,228.0,1


In [47]:
score=df3[colunas[0]]*0
for i in range(0,len(colunas)):
    score= score+betas[i]*df3[colunas[i]]

In [48]:
score

360362    0.000000
363722    0.000000
91437     0.207535
187871    0.000000
26306     0.000000
506185    0.000000
31707     0.000000
12262     0.629763
478536    0.000000
127968    0.207535
506588    0.000000
167947    0.000000
373875    0.000000
178294    0.000000
94387     0.629763
399901    0.629763
477156    0.000000
153174    0.629763
17942     0.000000
174017    0.000000
262780    0.000000
91972     0.629763
362194    0.629763
290937    0.629763
281515    0.000000
108719    0.000000
434440    0.000000
392345    0.207535
86655     0.000000
101821    0.000000
222520    0.000000
388996    0.000000
43879     0.000000
98944     0.207535
311542    0.000000
159415    0.000000
480911    0.000000
150692    0.000000
395688    0.207535
256181    0.000000
344518    0.207535
139700    0.000000
73954     0.000000
369346    0.000000
38760     0.207535
310196    0.000000
330282    0.000000
254893    0.000000
253437    0.000000
373299    0.000000
230078    0.000000
131449    0.000000
280754    0.

In [142]:
df_z.to_excel("output.xlsx")

Ks_2sampResult(statistic=1.0, pvalue=0.0)

In [49]:
#score
df_z['score']=df_z[colunas[0]]*0


In [50]:
betas

col1_a   -1.488686e+10
col1_b   -1.488686e+10
col2_a    1.488686e+10
dtype: float64

In [45]:
for i in range(0,len(betas)):
    df_z['score']= df_z['score']+betas[i]*df3[colunas[i]]

In [46]:
df_z

,PAGTO,col1_a,col1_b,col2_a,score
0,0,0,0,0,NaN
1,0,1,0,1,NaN
2,0,0,0,0,NaN
3,0,0,0,0,NaN
4,0,0,0,0,NaN
5,0,1,0,1,NaN
6,0,0,0,0,NaN
7,0,0,0,0,NaN
8,0,1,0,1,NaN
9,0,1,0,1,0.629763


In [345]:
round((np.exp(267) /(1+np.exp(267))*1000,0)

SyntaxError: unexpected EOF while parsing (<ipython-input-345-632c1163e8fc>, line 1)

In [ ]:
df_z['SCORE_1']=round((np.exp(df_z['SCORE']) /(1+np.exp(df_z['SCORE'])))*1000,0)

formula=PAGTO ~+col1_a
formula=PAGTO ~+col1_a+col1_b
formula=PAGTO ~+col1_a+col1_b+col1_c
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b+col3_c
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b+col3_c+col3_d
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b+col3_c+col3_d+col3_e
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b+col3_c+col3_d+col3_e+col4_a
formula=PAGTO ~+col1_a+col1_b+col1_c+col1_d+col1_e+col2_a+col2_b+col3_a+col3_b+col3_c+col3_d+col3_e+col4_a+col4_b
formula=PAGTO ~+col1_a+col1_b+col1_c+col1

In [ ]:
  formula = 'PAGTO ~'+teste[1]+'+'+teste[2]

In [118]:
colunas[1:]

Index(['col1_a', 'col1_b', 'col1_c', 'col1_d', 'col1_e', 'col2_a', 'col2_b', 'col3_a', 'col3_b', 'col3_c', 'col3_d', 'col3_e', 'col4_a', 'col4_b', 'col4_c', 'col5_a', 'col5_b', 'col5_c', 'col5_d', 'col6_a', 'col6_b', 'col6_c', 'col7_a', 'col7_b', 'col7_c', 'col7_d', 'col7_e', 'col7_f', 'col8_a', 'col8_b', 'col9_a', 'col9_b', 'col9_c', 'col9_d', 'col10_a', 'col10_b', 'col10_c', 'col11_a', 'col11_b', 'col11_c'], dtype='object')

In [300]:
formula = 'PAGTO ~ col1_a+	col1_b+	col1_c+	col1_d+	col1_e+	col2_a+	col2_b+col3_a+	col3_b+	col3_c+	col3_d+	col3_e+	col4_a+	col4_b+	col4_c+	col5_a+	col5_b+	col5_c+	col5_d+	col6_a+	col6_b+	col6_c+	col7_a+	col7_b+	col7_c+	col7_d+	col7_e+	col7_f+	col8_a+	col8_b+	col9_a+	col9_b+	col9_c+	col9_d+col10_a+	col10_b+col10_c+col11_a+col11_b+col11_c'

In [319]:
formula = 'PAGTO ~ col1_a+	col1_b+	col1_c+	col1_d+	col1_e+	col2_a+	col2_b+col3_a+	col3_b+	col3_c+	col3_d+	col3_e+	col4_a+	col4_b+	col4_c+	col5_a+	col5_b+	col5_c+	col5_d+	col6_a+	col6_b+	col6_c+	col7_a+	col7_b+	col7_c+	col7_d+	col7_e+	col7_f+	col8_a+	col8_b+	col9_a+	col9_b+	col9_c+	col9_d+col10_a+	col10_b+col10_c+col11_a+col11_b+col11_c'

In [ ]:
# tirar: col2_c, col5_e col6_d, col8_c ,col9_e e col10_d

In [150]:
model = smf.glm(formula = formula, data=df3, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9968
Model Family:                Binomial   Df Model:                           31
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Sun, 02 Feb 2020   Deviance:                          nan
Time:                        16:04:15   Pearson chi2:                 7.62e+18
No. Iterations:                   100   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.661e+21   1.22e+14  -4.65e+07      0.000   -5.66e+21   -5.66e+21
col1_a     -8.267e+20   1.78e+13  -4.65e+07      0.0

40


40

In [220]:
colunas

Index(['col1_a', 'col1_b', 'col1_c', 'col1_d', 'col1_e', 'col2_a', 'col2_b', 'col3_a', 'col3_b', 'col3_c', 'col3_d', 'col3_e', 'col4_a', 'col4_b', 'col4_c', 'col5_a', 'col5_b', 'col5_c', 'col5_d', 'col6_a', 'col6_b', 'col6_c', 'col7_a', 'col7_b', 'col7_c', 'col7_d', 'col7_e', 'col7_f', 'col8_a', 'col8_b', 'col9_a', 'col9_b', 'col9_c', 'col9_d', 'col10_a', 'col10_b', 'col10_c', 'col11_a', 'col11_b', 'col11_c'], dtype='object')

In [299]:
score.describe()

count    1.000000e+04
mean     5.660539e+21
std      6.223822e+14
min      5.660534e+21
25%      5.660538e+21
50%      5.660539e+21
75%      5.660539e+21
max      5.660546e+21
dtype: float64

In [275]:
#criar score zerado
score=df3[colunas[0]]*0

for i in range(0,len(colunas)):
    score= score+betas[i]*df3[colunas[i]]

In [276]:
score

271862    5.660539e+21
349230    5.660539e+21
432448    5.660540e+21
409369    5.660538e+21
499019    5.660539e+21
449189    5.660540e+21
287552    5.660539e+21
288435    5.660539e+21
335650    5.660539e+21
61579     5.660538e+21
362407    5.660540e+21
323901    5.660539e+21
457032    5.660539e+21
235886    5.660539e+21
28272     5.660539e+21
151635    5.660538e+21
164540    5.660538e+21
313393    5.660539e+21
459350    5.660539e+21
453118    5.660541e+21
298706    5.660539e+21
134425    5.660538e+21
119878    5.660537e+21
26717     5.660538e+21
308993    5.660539e+21
94933     5.660539e+21
482500    5.660538e+21
482701    5.660538e+21
153380    5.660539e+21
418278    5.660539e+21
203003    5.660539e+21
450043    5.660539e+21
186588    5.660539e+21
384165    5.660539e+21
348256    5.660539e+21
16677     5.660538e+21
97989     5.660538e+21
483556    5.660539e+21
351770    5.660539e+21
319458    5.660539e+21
346476    5.660537e+21
147778    5.660538e+21
168998    5.660534e+21
268460    5

In [249]:
betas

col1_a    -8.266942e+20
col1_b    -8.266935e+20
col1_c    -8.266948e+20
col1_d    -8.266949e+20
col1_e    -2.382815e+21
col2_a    -1.556121e+21
col2_b    -1.556121e+21
col3_a     1.779268e+21
col3_b     1.779268e+21
col3_c     1.779268e+21
col3_d     1.779268e+21
col3_e     1.779268e+21
col4_a     3.030327e+21
col4_b     3.030326e+21
col4_c     3.030327e+21
col5_a    -5.039271e+20
col5_b    -5.039266e+20
col5_c    -5.039263e+20
col5_d    -5.039265e+20
col6_a     5.013833e+15
col6_b     3.867681e+15
col6_c     3.751920e+15
col7_a     8.442656e+20
col7_b     8.442645e+20
col7_c     8.442663e+20
col7_d     8.442654e+20
col7_e     8.442655e+20
col7_f     8.442701e+20
col8_a     8.217998e+20
col8_b     8.217996e+20
col9_a    -3.103486e+20
col9_b    -3.103485e+20
col9_c    -3.103486e+20
col9_d    -3.103486e+20
col10_a   -7.519865e+18
col10_b   -7.519790e+18
col10_c   -7.519595e+18
col11_a    2.389486e+21
col11_b    2.389486e+21
col11_c    2.389485e+21
dtype: float64

In [250]:
df3.to_excel("output.xlsx")  

In [157]:
## Score
df3['score']=0
for i in colunas:
    
    df3['score']= df3['score']+betas[i]*df3[colunas[i]]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

475453   -0.000000e+00
37490    -0.000000e+00
116090   -0.000000e+00
444700   -0.000000e+00
500522   -0.000000e+00
142386   -0.000000e+00
407391   -0.000000e+00
378926   -0.000000e+00
217308   -8.266942e+20
376471   -0.000000e+00
258582   -8.266942e+20
430242   -0.000000e+00
293697   -8.266942e+20
157258   -0.000000e+00
320329   -0.000000e+00
226428   -0.000000e+00
63134    -0.000000e+00
125676   -0.000000e+00
70825    -0.000000e+00
278053   -8.266942e+20
304834   -0.000000e+00
170475   -0.000000e+00
14443    -0.000000e+00
243047   -0.000000e+00
133259   -0.000000e+00
77699    -0.000000e+00
476481   -0.000000e+00
30376    -0.000000e+00
358082   -0.000000e+00
389181   -0.000000e+00
296424   -0.000000e+00
222156   -0.000000e+00
324371   -0.000000e+00
351670   -0.000000e+00
385380   -0.000000e+00
475944   -0.000000e+00
58626    -0.000000e+00
377323   -0.000000e+00
141796   -0.000000e+00
460533   -0.000000e+00
218869   -0.000000e+00
388306   -0.000000e+00
414751   -0.000000e+00
62246    -0

In [146]:
colunas[0]

'col1_a'

In [147]:
df3[colunas[0]]

475453    0
37490     0
116090    0
444700    0
500522    0
142386    0
407391    0
378926    0
217308    1
376471    0
258582    1
430242    0
293697    1
157258    0
320329    0
226428    0
63134     0
125676    0
70825     0
278053    1
304834    0
170475    0
14443     0
243047    0
133259    0
77699     0
476481    0
30376     0
358082    0
389181    0
296424    0
222156    0
324371    0
351670    0
385380    0
475944    0
58626     0
377323    0
141796    0
460533    0
218869    0
388306    0
414751    0
62246     0
383439    0
82518     0
501745    0
233348    0
213956    0
40970     0
127666    0
213506    0
354456    1
218110    0
478031    0
444244    0
106238    0
261297    0
82429     0
45754     0
194025    0
259811    0
91830     0
413299    0
114274    0
452708    0
198448    0
5143      0
464666    0
4920      0
404140    0
277398    0
57579     0
281197    0
106087    0
401235    0
505100    0
382296    0
411557    0
216591    0
460229    0
185775    0
354048    0
4885

In [141]:
df3.columns[1]

'col1_a'

In [142]:
colunas

Index(['col1_a', 'col1_b', 'col1_c', 'col1_d', 'col1_e', 'col2_a', 'col2_b', 'col3_a', 'col3_b', 'col3_c', 'col3_d', 'col3_e', 'col4_a', 'col4_b', 'col4_c', 'col5_a', 'col5_b', 'col5_c', 'col5_d', 'col6_a', 'col6_b', 'col6_c', 'col7_a', 'col7_b', 'col7_c', 'col7_d', 'col7_e', 'col7_f', 'col8_a', 'col8_b', 'col9_a', 'col9_b', 'col9_c', 'col9_d', 'col10_a', 'col10_b', 'col10_c', 'col11_a', 'col11_b', 'col11_c'], dtype='object')

In [129]:
result.params

Intercept   -5.660539e+21
col1_a      -8.266942e+20
col1_b      -8.266935e+20
col1_c      -8.266948e+20
col1_d      -8.266949e+20
col1_e      -2.382815e+21
col2_a      -1.556121e+21
col2_b      -1.556121e+21
col3_a       1.779268e+21
col3_b       1.779268e+21
col3_c       1.779268e+21
col3_d       1.779268e+21
col3_e       1.779268e+21
col4_a       3.030327e+21
col4_b       3.030326e+21
col4_c       3.030327e+21
col5_a      -5.039271e+20
col5_b      -5.039266e+20
col5_c      -5.039263e+20
col5_d      -5.039265e+20
col6_a       5.013833e+15
col6_b       3.867681e+15
col6_c       3.751920e+15
col7_a       8.442656e+20
col7_b       8.442645e+20
col7_c       8.442663e+20
col7_d       8.442654e+20
col7_e       8.442655e+20
col7_f       8.442701e+20
col8_a       8.217998e+20
col8_b       8.217996e+20
col9_a      -3.103486e+20
col9_b      -3.103485e+20
col9_c      -3.103486e+20
col9_d      -3.103486e+20
col10_a     -7.519865e+18
col10_b     -7.519790e+18
col10_c     -7.519595e+18
col11_a     

-8.266942361086398e+20

In [69]:
teste=df3.columns

In [70]:
teste[1]

'col1_a'

In [71]:
teste

Index(['PAGTO', 'col1_a', 'col1_b', 'col1_c', 'col1_d', 'col1_e', 'col2_a',
       'col2_b', 'col2_c', 'col3_a', 'col3_b', 'col3_c', 'col3_d', 'col3_e',
       'col4_a', 'col4_b', 'col4_c', 'col5_a', 'col5_b', 'col5_c', 'col5_d',
       'col5_e', 'col6_a', 'col6_b', 'col6_c', 'col6_d', 'col7_a', 'col7_b',
       'col7_c', 'col7_d', 'col7_e', 'col7_f', 'col8_a', 'col8_b', 'col8_c',
       'col9_a', 'col9_b', 'col9_c', 'col9_d', 'col9_e', 'col10_a', 'col10_b',
       'col10_c', 'col10_d', 'col11_a', 'col11_b', 'col11_c'],
      dtype='object')

In [86]:
formula = 'PAGTO ~'+teste[1]+'+'+teste[2]

In [87]:
formula

'PAGTO ~col1_a+col1_b'

In [88]:
 model = smf.glm(formula = formula, data=df3, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  PAGTO   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4277.0
Date:                Sun, 02 Feb 2020   Deviance:                       8554.1
Time:                        13:26:56   Pearson chi2:                 1.00e+04
No. Iterations:                     4   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7811      0.031    -58.324      0.000      -1.841      -1.721
col1_a         0.5956      0.084      7.087      0.0

In [89]:
from sklearn.model_selection import train_test_split

In [ ]:
Xo_train, Xo_test, yo_train, yo_test = train_test_split(data_o, target_o, test_size=0.20, random_state=4) 